In [1]:
import xml.etree.cElementTree as ETree
import pprint
import re
import codecs
import json
import sys

In [2]:
execfile('osm_to_json.py')
inputfile = 'santiago.xml'

In [ ]:
structures, problems, street_names, tags1 = check_struct(inputfile)

In [ ]:
data, problems_data, tagtypes = process_map(inputfile)

In [5]:
tagtypes

{'acc': [{str}, 1],
 'access': [{str}, 2791],
 'addr:city': [{str, unicode}, 5499],
 'addr:country': [{str}, 3581],
 'addr:district': [{str}, 5],
 'addr:door': [{float}, 1],
 'addr:full': [{str, unicode}, 883],
 'addr:housename': [{float, str, unicode}, 374],
 'addr:housenumber': [{float, str, unicode}, 212906],
 'addr:interpolation': [{str}, 101575],
 'addr:postcode': [{float}, 206],
 'addr:state': [{str, unicode}, 107],
 'addr:street': [{float, str, unicode}, 214250],
 'addr:unit': [{str}, 1],
 'admin_level': [{float}, 135],
 'aerialway': [{str}, 336],
 'aerialway:occupancy': [{float}, 14],
 'aerodrome:type': [{str}, 1],
 'aeroway': [{str}, 267],
 'aiccess': [{str}, 3],
 'airmark': [{str}, 2],
 'alt_name': [{str, unicode}, 90],
 'amenity': [{str, unicode}, 9223],
 'area': [{str}, 209],
 'artist_name': [{unicode}, 1],
 'artwork_type': [{str}, 5],
 'atm': [{str}, 220],
 'attraction': [{str}, 1],
 'attribution': [{str, unicode}, 11454],
 'backrest': [{str}, 11],
 'barrier': [{str}, 7250

In [8]:
sorted(tagtypes.items(), key=lambda e: e[1][1])

[('fuente', [{str}, 1]),
 ('service:computer', [{str}, 1]),
 ('fax', [{str}, 1]),
 ('source:amenity', [{str}, 1]),
 ('name:uz', [{str}, 1]),
 ('name:ur', [{unicode}, 1]),
 ('recycling:magazines', [{str}, 1]),
 ('name:ug', [{str}, 1]),
 ('fireplace', [{str}, 1]),
 ('crop', [{str}, 1]),
 ('name:is', [{unicode}, 1]),
 ('service:press', [{str}, 1]),
 ('oneway:bycicle', [{str}, 1]),
 ('name:wikipedia', [{str}, 1]),
 ('name:bn', [{unicode}, 1]),
 ('name:bo', [{unicode}, 1]),
 ('web', [{str}, 1]),
 ('name:be', [{unicode}, 1]),
 ('name:bg', [{unicode}, 1]),
 ('fuel_service', [{str}, 1]),
 ('name:pl', [{str}, 1]),
 ('from', [{float}, 1]),
 ('name:pih', [{str}, 1]),
 ('service:copy', [{str}, 1]),
 ('have_riverbank', [{str}, 1]),
 ('capital', [{str}, 1]),
 ('name:scn', [{unicode}, 1]),
 ('name:vep', [{str}, 1]),
 ('name:pnb', [{unicode}, 1]),
 ('name:ace', [{unicode}, 1]),
 ('male', [{str}, 1]),
 ('inscription', [{unicode}, 1]),
 ('recycling:cooking_oil', [{str}, 1]),
 ('name:mr', [{unicode}, 1])

What do we want to check, given the problems we found out:

* Street names: we don't have the same street naming system, but some problems can be solved.
* Phone numbers: lately the phone number at Santiago were changed. We detected outdated or plainly wrong elements.
* E-mails: certainly bad emails
* Invalid types, or ambiguous values for keys
* Inconcistency in the use of 'addr:city'

## Validity: conforms to a schema

Mostly valid, in terms of the precessence of mandatory information for nodes, ways and relations. However, the validy of the 'tags' is under discussion.

* Mandatory fields
* Fields with unique values
* Valid references to other fields
* Cross-field consistency (addr:street with way:name)
* Range validity: phone numbers, postal codes, email.


## Accuracy: conforms to gold standard



## Completeness: are all records there?

## Consistency: matches other data

## Uniformity: same units

In [ ]:
street_names = []
components = []
for e in data:
    if e['type'] == 'way':
        if 'highway' in e.keys() and 'name' in e.keys():
            street_names.append(e['name'])
            components.append(e['name'].split(' ')[0])


In [ ]:
import pandas as pd
df_1 = pd.DataFrame(data={'name': street_names, 'num': 1})
df_2 = pd.DataFrame(data={'name': components, 'num': 1})

In [ ]:
df_1.groupby('name').count().reset_index().sort('name')

In [ ]:
db = get_db('dataw')

In [ ]:
phone = []
cursor = db.stgo.find({'phone': {'$exists': 1}})
for c in cursor:
    phone.append(c['phone'])

cursor = db.stgo.find({'contact:phone': {'$exists': 1}})
for c in cursor:
    phone.append(c['contact:phone'])

In [ ]:
phone